In [ ]:
# -----------------------------------------------------------------------------
# Customer Segmentation using K-Means Clustering
# -----------------------------------------------------------------------------
# This notebook performs customer segmentation on preprocessed marketing data.
# The goal is to identify distinct customer groups based on purchasing behavior,
# demographics, and engagement metrics to enable targeted marketing strategies.
#
# Business Context: 
# The company currently uses broad, non-personalized campaigns. By identifying
# natural customer segments, we can tailor marketing efforts, improve ROI,
# and increase campaign conversion rates.
# -----------------------------------------------------------------------------


In [ ]:
#  1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples

import warnings
warnings.filterwarnings('ignore')

# Set style for better visuals
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')


ModuleNotFoundError: No module named 'seaborn'